In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import math
import missingno as msno
import dateparser
import time
from numpy.core.numeric import NaN

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import matplotlib as mpl
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn import datasets, svm
from sklearn.model_selection import GridSearchCV

In [2]:
# import datasets
test = pd.read_csv("../../test.csv")
train = pd.read_csv("../../train.csv")


In [3]:
train = train.drop(columns=['Unnamed: 0'])
test = test.drop(columns=['Unnamed: 0'])

In [4]:
# Get the columns whch are only in the training set form the original file
train_orig = pd.read_csv("../../training_set_VU_DM.csv")

gross_bookings_usd = train_orig['gross_bookings_usd']
click_bool = train_orig['click_bool']
booking_bool = train_orig['booking_bool']
position = train_orig['position']

In [5]:
search_id = train['srch_id']
prop_id = train['prop_id']
prop_id

0             893
1           10404
2           21315
3           27348
4           29604
            ...  
4958342     77700
4958343     88083
4958344     94508
4958345    128360
4958346    134949
Name: prop_id, Length: 4958347, dtype: int64

In [6]:
results = pd.DataFrame()
results['SearchId'] = search_id
results['PropertyId'] = prop_id
results['Booking_bool'] = booking_bool
results['Probability_of_booking'] = [0] * len(train)
results

,SearchId,PropertyId,Booking_bool,Probability_of_booking
0,1,893,0,0
1,1,10404,0,0
2,1,21315,0,0
3,1,27348,0,0
4,1,29604,0,0
...,...,...,...,...
4958342,332785,77700,0,0
4958343,332785,88083,0,0
4958344,332785,94508,0,0
4958345,332785,128360,1,0


In [7]:
training_subset = train[:50000]
testing_subset = train[50000:60000]


training_results_subset = results[:50000]
testing_results_subset = results[50000:60000]

In [8]:
training_subset = training_subset.drop(columns=['srch_id', 'comp_percent', 'srch_query_affinity_score'])
testing_subset = testing_subset.drop(columns=['srch_id', 'comp_percent', 'srch_query_affinity_score'])

In [9]:
testing_results_subset

,SearchId,PropertyId,Booking_bool,Probability_of_booking
50000,3314,78605,0,0
50001,3314,82035,0,0
50002,3314,92123,0,0
50003,3314,105713,0,0
50004,3314,112415,0,0
...,...,...,...,...
59995,3997,67140,0,0
59996,3997,67619,0,0
59997,3997,72508,0,0
59998,3997,77444,0,0


In [10]:





# clf = RandomForestClassifier()
# print(clf.get_params().keys())

# # parameters = {'n_estimators':[200, 300, 400, 500,600,700, 800, 900], 'min_samples_split':[2,4,6,8,10,12], 'min_samples_leaf':[1,2,4,6,8]}

# # parameters = {'n_estimators':[200, 300], 'min_samples_split':[2,4]}

# # # class weight, niet gerelateerd, alle classes even zwaar wegen
# # clf = GridSearchCV(clf, parameters)

# # clf.fit(label_data,  np.ravel(surv_data))
# # clf.best_params_

# clf = RandomForestClassifier()
# # clf.fit(input_columns,  desired_outcome)
# result = clf.predict(col_test)

In [11]:
for var in training_subset:
    col = training_subset[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())


In [12]:
np.ravel(training_results_subset['Booking_bool'])

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [13]:
clf = RandomForestClassifier()
clf.fit(training_subset, np.ravel(training_results_subset['Booking_bool'])) 
predictions = clf.predict_proba(testing_subset)
predictions

array([[1.  , 0.  ],
       [1.  , 0.  ],
       [0.98, 0.02],
       ...,
       [0.93, 0.07],
       [0.84, 0.16],
       [0.94, 0.06]])

In [14]:
pred_1 = []
for i in range(len(predictions)):
    pred_1.append(predictions[i][1])

In [15]:
testing_results_subset['Probability_of_booking'] = pred_1

<ipython-input-15-423027d6e6c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_results_subset['Probability_of_booking'] = pred_1


In [16]:
testing_results_subset

,SearchId,PropertyId,Booking_bool,Probability_of_booking
50000,3314,78605,0,0.00
50001,3314,82035,0,0.00
50002,3314,92123,0,0.02
50003,3314,105713,0,0.00
50004,3314,112415,0,0.00
...,...,...,...,...
59995,3997,67140,0,0.08
59996,3997,67619,0,0.05
59997,3997,72508,0,0.07
59998,3997,77444,0,0.16


In [17]:
people = testing_results_subset['SearchId']
people

50000    3314
50001    3314
50002    3314
50003    3314
50004    3314
         ... 
59995    3997
59996    3997
59997    3997
59998    3997
59999    3997
Name: SearchId, Length: 10000, dtype: int64

In [18]:
x = testing_results_subset.loc[testing_results_subset['SearchId'] == 3994]
x

,SearchId,PropertyId,Booking_bool,Probability_of_booking
59941,3994,269,0,0.02
59942,3994,3274,0,0.05
59943,3994,7739,0,0.06
59944,3994,8425,0,0.11
59945,3994,9016,0,0.09
59946,3994,11368,0,0.05
59947,3994,13634,0,0.02
59948,3994,17408,0,0.07
59949,3994,17626,0,0.03
59950,3994,21851,0,0.07


# Andere aanpak
Het lijkt dat hij alles overfit op 'niet boeken'. Het idee is om veel hotels die neit worden geboekt te verwijderen uit de dataset

In [19]:
training_results_subset

,SearchId,PropertyId,Booking_bool,Probability_of_booking
0,1,893,0,0
1,1,10404,0,0
2,1,21315,0,0
3,1,27348,0,0
4,1,29604,0,0
...,...,...,...,...
49995,3314,60256,0,0
49996,3314,67451,0,0
49997,3314,71335,0,0
49998,3314,72141,0,0


In [20]:
hotel_bookings = list(training_results_subset['Booking_bool'])
print('in 50 hotels, ther are',hotel_bookings.count(0),'hotels which are not booked')
print('in 50 hotels, ther are',hotel_bookings.count(1),'hotels which are booked')
# Mkae the training set so that for every booked hotel there are 5 hotels which are not booked

in 50 hotels, ther are 48623 hotels which are not booked
in 50 hotels, ther are 1377 hotels which are booked


In [21]:
nr_negativehotel_remove = hotel_bookings.count(0) - 3*hotel_bookings.count(1)
nr_negativehotel_remove
# for i in range(nr_negativehotel_remove):
    
removed = 0
while removed != nr_negativehotel_remove:
    index = random.randint(0, len(training_results_subset)-1)
    if training_results_subset['Booking_bool'][index] == 0:
#         print(training_results_subset['Booking_bool'][index])
#         remove row from training and results list
#         training_results_subset.drop([index])
        training_results_subset.drop(training_results_subset.index[index], inplace=True)
        training_results_subset = training_results_subset.reset_index(drop=True)
        
#         training_subset.drop([index])
        training_subset.drop(training_subset.index[index], inplace=True)
        training_subset = training_subset.reset_index(drop=True)
        removed+=1  

C:\Users\Marlon\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [22]:
training_subset

,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,comp_rate,comp_inv,month,daypart
0,12,187,0.0,0.0,219,29604,4,3.5,1,2.5,...,4,0,1,1,8.0,1,0.000000,0.00,4,2
1,12,187,0.0,0.0,219,68914,2,3.0,1,2.0,...,4,0,1,1,8.0,1,0.000000,-0.25,4,2
2,5,219,0.0,0.0,219,22824,3,4.0,0,3.5,...,1,0,1,1,2.0,1,0.000000,0.00,12,2
3,5,219,0.0,0.0,219,137826,2,3.0,0,3.5,...,1,0,1,1,2.0,1,0.000000,0.00,12,2
4,14,100,0.0,0.0,100,22135,0,5.0,0,2.0,...,2,0,1,0,7.0,0,0.000000,0.00,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5503,5,219,0.0,0.0,219,118800,3,3.5,0,3.5,...,6,2,2,0,1.0,0,-0.500000,0.00,3,2
5504,5,219,0.0,0.0,219,121785,3,4.0,0,3.5,...,6,2,2,0,1.0,0,0.000000,0.00,3,2
5505,5,219,0.0,0.0,219,61296,4,4.0,0,2.5,...,1,0,1,0,12.0,1,0.000000,1.00,1,2
5506,5,219,0.0,0.0,219,125993,3,2.5,0,2.5,...,1,0,1,0,12.0,1,1.000000,0.00,1,2


In [23]:
hotel_bookings = list(training_results_subset['Booking_bool'])
print('in 50 hotels, ther are',hotel_bookings.count(0),'hotels which are not booked')
print('in 50 hotels, ther are',hotel_bookings.count(1),'hotels which are booked')

in 50 hotels, ther are 4131 hotels which are not booked
in 50 hotels, ther are 1377 hotels which are booked


In [24]:
clf = RandomForestClassifier()
clf.fit(training_subset, np.ravel(training_results_subset['Booking_bool'])) 
predictions = clf.predict_proba(testing_subset)
predictions

array([[0.92, 0.08],
       [0.9 , 0.1 ],
       [0.9 , 0.1 ],
       ...,
       [0.82, 0.18],
       [0.63, 0.37],
       [0.69, 0.31]])

In [25]:
pred_1 = []
for i in range(len(predictions)):
    pred_1.append(predictions[i][1])
    
testing_results_subset['Probability_of_booking'] = pred_1

<ipython-input-25-da499298fc8b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_results_subset['Probability_of_booking'] = pred_1


In [26]:
testing_results_subset

,SearchId,PropertyId,Booking_bool,Probability_of_booking
50000,3314,78605,0,0.08
50001,3314,82035,0,0.10
50002,3314,92123,0,0.10
50003,3314,105713,0,0.05
50004,3314,112415,0,0.10
...,...,...,...,...
59995,3997,67140,0,0.21
59996,3997,67619,0,0.13
59997,3997,72508,0,0.18
59998,3997,77444,0,0.37


In [27]:
x = testing_results_subset.loc[testing_results_subset['SearchId'] == 3333]
x

,SearchId,PropertyId,Booking_bool,Probability_of_booking
50351,3333,2136,0,0.35
50352,3333,20114,0,0.40
50353,3333,24067,0,0.27
50354,3333,30890,0,0.37
50355,3333,32464,0,0.28
50356,3333,36142,0,0.36
50357,3333,41325,0,0.34
50358,3333,48167,0,0.30
50359,3333,56214,0,0.21
50360,3333,59572,0,0.32
